## Pre-processing

In [211]:
import pandas as pd
import numpy as np

In [212]:
# import preprocessing routine
from preprocess_module import fraud_preprocessor

# call preprocessor 
full_df = fraud_preprocessor(i_flag=0)

## Feature Engineering

In [213]:
# create physician count column 
phys_count = full_df.groupby(['AttendingPhysician', 'Provider'])['AttendingPhysician'].count()\
           .reset_index(name='Phys_Count').groupby(['Provider'])['Phys_Count'].count().reset_index()


In [214]:
# create patient count column 
patient_count = full_df.groupby(['Provider', 'BeneID'])['BeneID'].count().reset_index(name='Patient_Count')\
                .groupby('Provider')['Patient_Count'].count().reset_index()

features1 = pd.merge(phys_count, patient_count, on='Provider')

In [215]:
# create claim count column 
claim_count = full_df.groupby(['Provider', 'ClaimID'])['ClaimID'].count().reset_index(name='Claim_Count').\
         groupby('Provider')['Claim_Count'].count().reset_index()

features2 = pd.merge(features1, claim_count, on='Provider')

In [216]:
##create service type column
prov_type = full_df.groupby(['Provider', 'PatientType'])['BeneID'].count().reset_index(name='').drop('', axis=1)
prov_type.head()

# create a dictionary provider by service type
lst_prov_type = list(zip(prov_type['Provider'], prov_type['PatientType']))

# feed in empty dict with values inpatient, outpatient, or both
prov_type_dict = {}
for i in lst_prov_type:
    if i[0] not in prov_type_dict:
        prov_type_dict[i[0]]= i[1]    
    else:
        prov_type_dict[i[0]] = 'Both'
        
# create new column service type by povider
service_type = pd.DataFrame(prov_type_dict.keys(), prov_type_dict.values()).reset_index().\
               rename(columns={'index':'Service_Type', 0:'Provider'})

# add service type column 
features3 = pd.merge(features2, service_type, on='Provider')

In [217]:
# create new column with count of conditions
cond_count = full_df.filter(regex='Chron').replace(to_replace=2, value=0).sum(axis=1).\
             reset_index(name='Cond_Count')

# add provider and claimID
cond_count = pd.concat([cond_count, full_df[['Provider', 'ClaimID']]], axis=1)

# create avg cond and avg claim columns
avg_cond_claim = cond_count.groupby(['Provider', 'Cond_Count'])['ClaimID'].count().reset_index(name='Claim_Count')\
                 .groupby('Provider').agg({'Cond_Count': 'mean', 'Claim_Count': 'mean'}).reset_index()\
                 .rename(columns={'Cond_Count':'Avg_Cond', 'Claim_Count': 'Avg_Claim'}).round() 

# add avg cond and avg claim columns
features4 = pd.merge(features3, avg_cond_claim, on='Provider')

In [220]:
# reformat claim start date column and DOB column 
claim_start = full_df['ClaimStartDt'].apply(pd.to_datetime, errors='coerce', format='%Y-%m-%d')
birth_date = full_df['DOB']

# create Age column of patient at time of claim
full_df['Age'] = claim_start - birth_date

#turn column in Years
full_df['Age'] = full_df['Age'] / np.timedelta64(1, 'Y')

#create average age by provider column 
avg_age = full_df.groupby(['Provider', 'BeneID'])['Age'].mean().reset_index(name = "Avg_Age").dropna()\
          .groupby('Provider')['Avg_Age'].mean().reset_index()

#convert avg_age column into integer
avg_age['Avg_Age'] = avg_age['Avg_Age'].astype(int)

# add avg age age column to feature data set
features5 = pd.merge(features4, avg_age, on='Provider')

In [221]:
#import train label df
label_train_df = pd.read_csv('Train-1542865627584.csv')

#add label column
train_features = pd.merge(features5, label_train_df, on='Provider')
train_features.head()

,Provider,Phys_Count,Patient_Count,Claim_Count,Service_Type,Avg_Cond,Avg_Claim,Avg_Age,PotentialFraud
0,PRV51001,14,24,25,Both,6.0,4.0,78,No
1,PRV51003,44,117,132,Both,5.0,19.0,68,Yes
2,PRV51004,38,138,149,Outpatient,4.0,23.0,72,No
3,PRV51005,7,495,1165,Outpatient,4.0,154.0,70,Yes
4,PRV51007,10,58,72,Both,4.0,12.0,67,No
